<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_CB_I_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_CB-I',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/510Normal.png', '/content/Gold_Folder_Final/Test_Dat

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_CB-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_CB-I/510Normal.png', '/content/Gold_Folder_Final/Test_Dat

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 674ms/step - kl_loss: 2257483.0000 - loss: 2303511.7500 - reconstruction_loss: 46028.7656
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - kl_loss: 29.9565 - loss: 43652.8594 - reconstruction_loss: 43622.9023
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - kl_loss: 217.3768 - loss: 36271.0703 - reconstruction_loss: 36053.6953
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - kl_loss: 100.7845 - loss: 29409.2598 - reconstruction_loss: 29308.4766
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - kl_loss: 115.5065 - loss: 27289.8613 - reconstruction_loss: 27174.3555
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - kl_loss: 126.0466 - loss: 26420.5762 - reconstruction_loss: 26294.5312
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - kl_loss: 149.6344 - loss: 24315.9043 - reconstruction_loss: 24166.2715
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - kl_loss: 149.4053 - loss: 22424.9629 - reconstruction_loss: 22275.5586


In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_CB-I',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=13feadd9b260e0f4aa8b05c6bbdc18bc1c46c6f7ce6909a33e42df09d0695585
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.0044846554653024005
78.46161
[[ 8.311858   4.653488  -7.8071156]]
[[-1.0563024 -3.0214791 -3.7320268]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.002421201745164706
83.6872
[[ 9.261497   4.0499606 -7.713615 ]]
[[-1.4001738 -2.913453  -4.0654135]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.004110647631783045
81.18922
[[ 8.409088  4.786333 -7.962621]]
[[-1.0579001 -3.091964  -3.7882507]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.003344942070441809
85.82882
[[ 9.559376   3.879005  -7.7149158]]
[[-1.4371122 -2.902503  -4.0600114]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0025883272585236652
84.75296
[[ 9.305426   4.0946636 -7.7725253]]
[[-1.3966526 -2.9385173 -4.0841727]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.002766750486135207
83.472595
[[ 8.873917   4.4790125 -7.911669 ]]
[[-1.2123184 -3.0208504 -3.9439285]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.002672468446587138
85.07829
[[ 9.353401   4.0667906 -7.7709723]]
[[-1.405996  -2.9382486 -4.0845237]]
1/1 ━━━━━

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('CB-I_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.004804853589723262
84.08288
[[ 8.864968   4.537753  -7.9715557]]
[[-1.1284423 -3.0666656 -3.8547885]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0050545648142796586
84.68086
[[ 8.901079  4.546172 -8.001745]]
[[-1.117783  -3.0802808 -3.8445063]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0043089165859609094
81.00276
[[ 8.425362   4.749413  -7.9500976]]
[[-1.0142974 -3.0695412 -3.7411602]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.004236784233674456
81.693344
[[ 8.496616   4.7326813 -7.967976 ]]
[[-1.0406092 -3.072284  -3.7718287]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.004062964870377883
82.86134
[[ 8.636984   4.6843963 -7.9863753]]
[[-1.0876284 -3.0758374 -3.8311179]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.004050375563193653
82.98936
[[ 8.587122  4.744413 -8.022955]]
[[-1.0480188 -3.0956223 -3.8009286]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.004344897107195118
81.77225
[[ 8.591201   4.638502  -7.9300337]]
[[-1.0752362 -3.0553975 -3.7931507]]
1/1 ━━━━━

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('CB-I_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
0.15307648379645655
61.77984
[[ 6.706513   4.6361575 -7.3365645]]
[[ 0.7471322 -3.6678858 -0.7185334]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
0.030563416978784702
92.37656
[[ 9.5748625  4.382909  -8.300038 ]]
[[-0.8444971 -3.194788  -3.4322376]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.14874624664311006
60.87366
[[ 6.7778554  4.5043793 -7.238505 ]]
[[ 0.63924503 -3.5666246  -0.84038675]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.002855456446290358
83.27496
[[ 9.125274   4.177418  -7.7584186]]
[[-1.3410408 -2.9447885 -4.0174193]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
0.17625844969860274
60.591
[[ 6.5619245  4.6605043 -7.286973 ]]
[[ 0.79158556 -3.7040489  -0.622043  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
0.16323397219753735
61.715134
[[ 6.631534   4.6971574 -7.352036 ]]
[[ 0.79122376 -3.7306287  -0.64112484]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
0.0027327205976008556
82.46666
[[ 9.164145   4.054081  -7.6740217]]
[[-1.3726122 -2.9034576 -4.0233455]]
1

In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('CB-I_testanom.csv')

In [ ]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>